In [1]:
import numpy as np
import math
import pandas as pd
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

import time
import holidays
import dateutil.easter as easter

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from xgboost import XGBRegressor

import catboost
from catboost import CatBoostRegressor, Pool

In [2]:
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [3]:
train['date'] , test['date'] = (pd.to_datetime(train['date'])),(pd.to_datetime(test['date']))

In [4]:
#Adding holiday info
def add_holiday(train):
    finland_holiday = holidays.CountryHoliday('FI',years = list(range(2015,2022)))
    norway_holiday = holidays.CountryHoliday('NO',years = list(range(2015,2022)))
    sweden_holiday = holidays.CountryHoliday('SE',years = list(range(2015,2022)))

    holiday = finland_holiday.copy()
    holiday.update(norway_holiday)
    holiday.update(sweden_holiday)

    train['holiday_name'] = train['date'].map(holiday)
    train['holiday'] = np.where(train['holiday_name'].isnull(),0,1)
    train.drop(['holiday_name' , 'row_id'],axis = 1,inplace = True)

    test['holiday_name'] = test['date'].map(holiday)
    test['holiday'] = np.where(test['holiday_name'].isnull(),0,1)
    test.drop(['holiday_name' , 'row_id'],axis = 1,inplace = True)

    print("Holidays added successfully")
    return train,test

train,test = add_holiday(train)

Holidays added successfully


In [5]:
#more feature addition
gdp_exp = 1.2121103201489674
gdp_df = pd.read_csv('train.csv')
gdp_df.set_index('year',inplace = True)
def get_gdp(row):
    country = 'GDP_'+row.country
    return gdp_df.loc[row.date.year,country]

train['gdp'] = np.lop1p(train.apply(get_gdp,axis =1))
test['gdp'] = np.lop1p(test.apply(get_gdp,axis =1))
    

KeyError: "None of ['year'] are in the columns"

In [ ]:
gd